# Imports

In [ ]:
import pandas as pd
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
export_dir = os.getcwd()
from pathlib import Path
import pickle
from collections import defaultdict
import time
import torch
import torch.nn as nn
import copy
import optuna
import logging
import matplotlib.pyplot as plt
import random
import ipynb
import wandb
import importlib
from os import path

In [ ]:
data_name = "ML1M" ### Can be ML1M, Yahoo, Pinterest
recommender_name = "MLP" ## Can be MLP, VAE
DP_DIR = Path("processed_data", data_name) 
export_dir = Path(os.getcwd())
files_path = Path(export_dir.parent, DP_DIR)
checkpoints_path = Path(export_dir, "checkpoints")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
output_type_dict = {
    "VAE":"multiple",
    "MLP":"single"
}

num_users_dict = {
    "ML1M":6037,
    "Yahoo":13797, 
    "Pinterest":19155
}

num_items_dict = {
    "ML1M":3381,
    "Yahoo":4604, 
    "Pinterest":9362
}



recommender_path_dict = {
    #("ML1M","VAE"): Path(checkpoints_path, "VAE_ML1M_0.0007_128_10.pt"),
    ("ML1M","VAE"): Path(checkpoints_path, "VAE_ML1M_0.0003_64.pt"),
    ("ML1M","MLP"):Path(checkpoints_path, "MLP1_ML1M_0.0076_256_7.pt"),
    
    #("Yahoo","VAE"): Path(checkpoints_path, "VAE_Yahoo_0.0001_128_13.pt"),
    ("Yahoo","VAE"): Path(checkpoints_path, "VAE_Yahoo_128.pt"),

    ("Yahoo","MLP"):Path(checkpoints_path, "MLP2_Yahoo_0.0083_128_1.pt"),

    ("Pinterest","VAE"): Path(checkpoints_path, "VAE_Pinterest_0.0002_32_12.pt"),
    ("Pinterest","MLP"):Path(checkpoints_path, "MLP_Pinterest_0.0062_512_21_0.pt")
    
}


hidden_dim_dict = {
    #("ML1M","VAE"): None,
    ("ML1M","VAE"): [256,64],
    ("ML1M","MLP"): 32,

    ("Yahoo","VAE"): None,
    ("Yahoo","MLP"):32,
    
    ("Pinterest","VAE"): None,
    ("Pinterest","MLP"):512

}

Populairty_filtering=False

In [ ]:
output_type = output_type_dict[recommender_name] ### Can be single, multiple
num_users = num_users_dict[data_name] 
num_items = num_items_dict[data_name] 
hidden_dim = hidden_dim_dict[(data_name,recommender_name)]
recommender_path = recommender_path_dict[(data_name,recommender_name)]

## Data imports and preprocessing

In [ ]:
train_data = pd.read_csv(Path(files_path,f'train_data_{data_name}.csv'), index_col=0)
test_data = pd.read_csv(Path(files_path,f'test_data_{data_name}.csv'), index_col=0)
train_data['user_id'] = train_data.index
test_data['user_id'] = test_data.index
static_test_data = pd.read_csv(Path(files_path,f'static_test_data_{data_name}.csv'), index_col=0)

with open(Path(files_path,f'pop_dict_{data_name}.pkl'), 'rb') as f:
    pop_dict = pickle.load(f)

######## Removing most popular itemd from training and testing datasets for Filter_pop baseline
if (Populairty_filtering ==True): 
    sorted_dict = dict(sorted(pop_dict.items(), key=lambda item: item[1],reverse=True))
    Top_pop_items=list(sorted_dict.keys())[0:500]
    for i in Top_pop_items:
        train_data.iloc[:,i]=0
        test_data.iloc[:,i]=0
        static_test_data.iloc[:,i]=0

train_array = train_data.to_numpy()
test_array = test_data.to_numpy()
items_array = np.eye(num_items)
all_items_tensor = torch.Tensor(items_array).to(device)

In [ ]:
pop_array = np.zeros(len(pop_dict))
for key, value in pop_dict.items():
    pop_array[key] = value

In [ ]:
kw_dict = {'device':device,
          'num_items': num_items,
          'pop_array':pop_array,
          'all_items_tensor':all_items_tensor,
          'static_test_data':static_test_data,
          'items_array':items_array,
          'output_type':output_type,

          'recommender_name':recommender_name}

# Recommenders Architecture

In [ ]:
from ipynb.fs.defs.recommenders_architecture import *
importlib.reload(ipynb.fs.defs.recommenders_architecture)
from ipynb.fs.defs.recommenders_architecture import *


 # VAE Config (for VAE model)

In [ ]:
VAE_config = { "enc_dims": [256, 64], "dropout": 0.5, "anneal_cap": 0.2, "total_anneal_steps": 200000 }


# Loading recommender systems

In [ ]:
def load_recommender():
    if recommender_name=='MLP':
        recommender = MLP(hidden_dim, **kw_dict)
    elif recommender_name=='VAE':
        recommender = VAE(VAE_config, **kw_dict)
    recommender_checkpoint = torch.load(Path(checkpoints_path, recommender_path), map_location=torch.device('cpu'))
    recommender.load_state_dict(recommender_checkpoint)
    recommender.eval()
    for param in recommender.parameters():
        param.requires_grad= False
    return recommender
    
recommender = load_recommender()

# Help functions

In [ ]:
from ipynb.fs.defs.help_functions import *
importlib.reload(ipynb.fs.defs.help_functions)
from ipynb.fs.defs.help_functions import *

In [ ]:
def get_user_recommended_item(user_tensor, recommender, **kw):
    all_items_tensor = kw['all_items_tensor']
    num_items = kw['num_items']
    user_res = recommender_run(user_tensor, recommender, all_items_tensor, None, 'vector', **kw)[:num_items]
    user_tensor = user_tensor[:num_items]
    user_catalog = torch.ones_like(user_tensor)-user_tensor
    user_recommenations = torch.mul(user_res, user_catalog)
    # Get the indices of the items sorted by their recommendation score in descending order
    sorted_recommendations = torch.argsort(user_recommenations, descending=True)
    return(sorted_recommendations[0:10])

## Load / create top recommended items dict

In [ ]:
## Load / create top recommended items dict

create_dicts = True
if create_dicts:

    ## target and comparative items for training 
    targ_train, cmp_train = {}, {}
    ## target and comparative items for testing 
    targ_test, cmp_test = {}, {}
   
    for i in range(train_array.shape[0]):
        user_index = train_array[i][-1]
        user_tensor = torch.Tensor(train_array[i][:-1]).to(device)
        recomm_list=get_user_recommended_item(user_tensor, recommender, **kw_dict)
        
        ## Target item as the first item rank
        targ_train[user_index] = int(recomm_list[0])
        
        ## Sampling for the comparative item below the target item
        cmp_train[user_index]= np.array(recomm_list[1:9].cpu())

    for i in range(test_array.shape[0]):
        user_index = test_array[i][-1]
        user_tensor = torch.Tensor(test_array[i][:-1]).to(device)
        recomm_list=get_user_recommended_item(user_tensor, recommender, **kw_dict)
        
        ## Target item as the first item rank
        targ_test[user_index] = int(recomm_list[0])
        
        ## Sampling for the comparative item below the target item
        cmp_test[user_index] = np.array(recomm_list[1:9].cpu())

    with open(Path(files_path,f'targ_train_{data_name}_{recommender_name}.pkl'), 'wb') as f:
        pickle.dump(targ_train, f)
    with open(Path(files_path,f'cmp_train_{data_name}_{recommender_name}.pkl'), 'wb') as f:
        pickle.dump(cmp_train, f)
    with open(Path(files_path,f'targ_test_{data_name}_{recommender_name}.pkl'), 'wb') as f:
        pickle.dump(targ_test, f)
    with open(Path(files_path,f'cmp_test_{data_name}_{recommender_name}.pkl'), 'wb') as f:
        pickle.dump(cmp_test, f)



else:
    with open(Path(files_path,f'targ_train_{data_name}_{recommender_name}.pkl'), 'rb') as f:
        targ_train = pickle.load(f)
    with open(Path(files_path,f'cmp_train_{data_name}_{recommender_name}.pkl'), 'rb') as f:
        cmp_train = pickle.load(f)
    with open(Path(files_path,f'targ_test_{data_name}_{recommender_name}.pkl'), 'rb') as f:
        targ_test = pickle.load(f)
    with open(Path(files_path,f'cmp_test_{data_name}_{recommender_name}.pkl'), 'rb') as f:
        cmp_test = pickle.load(f)

# Explinaer Architecture

In [ ]:
class Explainer(nn.Module):
    def __init__(self, user_size, item_size, hidden_size):
        super(Explainer, self).__init__()
        
        self.users_fc = nn.Linear(in_features = user_size, out_features=hidden_size).to(device)
        self.items_fc = nn.Linear(in_features = item_size, out_features=hidden_size).to(device)
        self.bottleneck = nn.Sequential(
            nn.Tanh(),
            nn.Linear(in_features = hidden_size*2, out_features=hidden_size).to(device),
            nn.Tanh(),
            nn.Linear(in_features = hidden_size, out_features=user_size).to(device),
            nn.Sigmoid()
        ).to(device)
        
        
    def forward(self, user_tensor, item_tensor):
        user_output = self.users_fc(user_tensor.float())
        item_output = self.items_fc(item_tensor.float())
        combined_output = torch.cat((user_output, item_output), dim=-1)
        expl_scores = self.bottleneck(combined_output).to(device)
        
        return expl_scores
        

In [ ]:
def find_CLXR_mask(user_tensor, i1_tensor,i2_tensor,explainer):
    
    ## Mask for target item
    m1 = explainer(user_tensor, i1_tensor)
    ## Mask for comparative item
    m2 = explainer(user_tensor, i2_tensor)
    ## Contrastive mask (m1*(1-m2))
    m3 =(1-m2)*m1

    ## Multiplying users profile by the contrastive mask
    x_m3 = user_tensor* m3 

    
    m3_dict = {i: x_m3[i].item() for i in range(len(x_m3))}   

    return m3_dict

# CLXR Loss function

In [ ]:
class CLXR_loss(nn.Module):

    def __init__(self, lambda_pos, lambda_neg,lambda_cmp_m1, lambda_cmp_m3, lambda_cmp_neg, alpha):
        super(CLXR_loss, self).__init__()
        
        self.lambda_pos = lambda_pos
        self.lambda_neg = lambda_neg

        self.lambda_cmp_m1= lambda_cmp_m1
        self.lambda_cmp_m3= lambda_cmp_m3
        self.lambda_cmp_neg=lambda_cmp_neg
        
        self.alpha = alpha

        
        
    def forward(self, user_tensors, i1_tensors, i2_tensors, i1_id, i2_id, m1, m2, m3):

        
        neg_m1 = torch.sub(torch.ones_like(m1), m1)
        
        neg_m3 = torch.sub(torch.ones_like(m3), m3)
        
        xm1_pos = user_tensors * m1
        
        xm2_pos = user_tensors * m2
        
        xm3_pos= user_tensors * (m3)

        xm1_neg = user_tensors * neg_m1
        
        xm3_neg= user_tensors * neg_m3

        if output_type=='single':
            
            ## Recommeder output for the target item by applying m1 mask (m1 is the mask for the target item)
            y1_m1_pos = torch.diag(recommender_run(xm1_pos, recommender, i1_tensors, item_id=i1_id, wanted_output = 'single', **kw_dict))

             ## Negative mask version for the target item
            y1_m1_neg = torch.diag(recommender_run(xm1_neg, recommender, i1_tensors, item_id=i1_id, wanted_output = 'single', **kw_dict))
            
            ## Recommeder output for the comparative item by applying m2 mask (m2 is the mask for the comparative item)
            y2_m2_pos = torch.diag(recommender_run(xm2_pos, recommender, i2_tensors, item_id=i2_id, wanted_output = 'single', **kw_dict))

            ## Recommeder outputs for the target and comparative items by masking m3=m1*(1-m2) (comparatvie mask). target and comparative outputs for the comparative mask
            y1_m3_pos= torch.diag(recommender_run(xm3_pos, recommender, i1_tensors, item_id=i1_id, wanted_output = 'single', **kw_dict))
            y2_m3_pos= torch.diag(recommender_run(xm3_pos, recommender, i2_tensors, item_id=i2_id, wanted_output = 'single', **kw_dict))

           
            ## Negative comparative mask for the comparative item
            y2_m3_neg= torch.diag(recommender_run(xm3_neg, recommender, i2_tensors, item_id=i2_id, wanted_output = 'single', **kw_dict))

        else:
            
            ### target item output
            y1_m1_pos = recommender_run(xm1_pos, recommender, i1_tensors, item_id=i1_id, wanted_output = 'vector', **kw_dict)
            y1_m1_neg = recommender_run(xm1_neg, recommender, i1_tensors, item_id=i1_id, wanted_output = 'vector', **kw_dict)

            ### comparative item
            y2_m2_pos= recommender_run(xm2_pos, recommender, i2_tensors, item_id=i2_id, wanted_output = 'vector', **kw_dict)

            
            rows1=torch.arange(len(i1_id))
            rows2=torch.arange(len(i2_id))

            y1_m1_pos = y1_m1_pos[rows1, i1_id] 
            y1_m1_neg = y1_m1_neg[rows1, i1_id] 
            y2_m2_pos = y2_m2_pos[rows2, i2_id] 


            y1_m3_pos= recommender_run(xm3_pos, recommender, i1_tensors, item_id=i1_id, wanted_output = 'vector', **kw_dict)
            y2_m3_pos= recommender_run(xm3_pos, recommender, i2_tensors, item_id=i2_id, wanted_output = 'vector', **kw_dict)
            
            y1_m3_pos=y1_m3_pos[rows1, i1_id] 
            y2_m3_pos=y2_m3_pos[rows2, i2_id] 

            
            y2_m3_neg=recommender_run(xm3_neg, recommender, i2_tensors, item_id=i2_id, wanted_output = 'vector', **kw_dict)
            y2_m3_neg=y2_m3_neg[rows2, i2_id] 
            
          

        ## First loss term  ( maximizing rating score of the target item  for the positive and negative masks (same as LXR)   )
        pos_loss = - self.lambda_pos *torch.mean(torch.log(y1_m1_pos))
        neg_loss = self.lambda_neg * torch.mean(torch.log(y1_m1_neg))

        ## Second Loss term, comparative term ( minimizing the distance between the target and contarstive items  rating scores by masking m1 )
        cmp_loss_m1 = - self.lambda_cmp_m1 * torch.mean(torch.log(1+torch.exp(-(y1_m1_pos-y2_m2_pos))))
        cmp_loss_m3 = - self.lambda_cmp_m3 * torch.mean(torch.log(1+torch.exp(y1_m3_pos-y2_m3_pos)))
        cmp_loss_neg = self.lambda_cmp_neg * torch.mean(torch.log(torch.exp(y2_m3_neg))) 


        ## Third term ( sparsity terms )
        l = self.alpha * xm1_pos[user_tensors>0].mean() + self.alpha * xm3_pos[user_tensors>0].mean()



        
        ### combined loss (summing up all the terms
        combined_loss = pos_loss + neg_loss  + cmp_loss_m1+ cmp_loss_m3 +  cmp_loss_neg  + l        

        return combined_loss

In [ ]:
# i1: Id of target item
# i1_tensors: convert target id to an one-hot tensor
# i2: Id of comparative item  
#i1_tensors: convert comparative id to an one-hot tensor

def calculate_pos_neg_k(user_tensor, i1, i2, i1_tensors, i2_tensors, num_of_bins, explainer, k):
    
    user_hist_size = int(torch.sum(user_tensor))
    bins = [0] + [len(x) for x in np.array_split(np.arange(user_hist_size), num_of_bins, axis=0)]


    # Helper function to mask items
    def mask_items(user_tensor, sorted_m3, total_items, device):
        mask = torch.zeros_like(user_tensor, dtype=torch.float32, device=device)
        indices = [item[0] for item in sorted_m3[:total_items]]
        mask[indices] = 1
        return user_tensor - mask

    # Process each item set (first, second, comparative)
    def process_sim_items(m3, i1, i2, bins, user_tensor, user_hist_size, recommender, **kw_dict):
      
        sorted_m3 = list(sorted(m3.items(), key=lambda item: item[1], reverse=True))[:user_hist_size]

        total_items = 0
        for index, bin_size in enumerate(bins):
            total_items += bin_size

            ## Perturbations
            p = mask_items(user_tensor, sorted_m3, total_items, device)


            ## Rank of target item and comparative item by masking m3
            i1_rank = get_index_in_the_list(p, user_tensor, i1, recommender, **kw_dict) + 1
            i2_rank = get_index_in_the_list(p, user_tensor, i2, recommender, **kw_dict) + 1
            
            if (i1_rank > i2_rank):
                
                return total_items, index   ## returning the perturbation that lead to reverse target and comparative items and also index of bin
            
          
    
        return None, None

    # Find CLXR masks (m3)
    m3 = find_CLXR_mask(
        user_tensor, i1_tensors, i2_tensors, explainer)
    
    #### total items to be removed from the users profile to have reversion of ranking target item and contrastive item
    total_items, bin_index=process_sim_items(m3, i1, i2, bins, user_tensor, user_hist_size, recommender, **kw_dict)
    
  
    return total_items, bin_index
    

# CLXR training


In [ ]:

torch.manual_seed(42)
np.random.seed(42)

num_of_rand_users = 700 # number of users for evaluations
random_rows = np.random.choice(test_array.shape[0], num_of_rand_users, replace=False)
random_sampled_array = test_array[random_rows]

def clxr_training(trial):

    
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.01)

    ## Hyperparameters for the first term of loss
    lambda_neg = trial.suggest_float('lambda_neg', 0,50)

    lambda_pos = trial.suggest_float('lambda_pos', 0,50)

    ## Hyperparameters for the second term of loss
    lambda_cmp_m1 = trial.suggest_float('lambda_cmp_m1', 0,50)   # For the m1 mask of contrastive term (2nd term)
    lambda_cmp_m3 = trial.suggest_float('lambda_cmp_m3', 0,50)   # For the m3 mask of contrastive term (2nd term)
    lambda_cmp_neg =  trial.suggest_float('lambda_cmp_neg', 0,50)   # For the m3 (negative part) mask of contrastive term (2nd term)

    
    # Hyperparameter for the third term of loss (sparsity term)
    alpha = trial.suggest_categorical('alpha', [1]) # set alpha to be 1, change other hyperparameters

    batch_size = trial.suggest_categorical('batch_size', [32,64,128,256])

    explainer_hidden_size = trial.suggest_categorical('explainer_hidden_size', [32,64,128])

    epochs = 50
    
    wandb.init(
        project=f"{data_name}_{recommender_name}_CLXR_training",
        name=f"trial_{trial.number}",
        config={
        'learning_rate' : learning_rate,
        'alpha' : alpha,
        'lambda_neg' : lambda_neg,
        'lambda_pos' : lambda_pos,
        'lambda_cmp_m1' : lambda_cmp_m1,
        'lambda_cmp_m3' : lambda_cmp_m3,
        'lambda_cmp_neg' : lambda_cmp_neg,
        'batch_size' : batch_size,
        'explainer_hidden_size' : explainer_hidden_size,
        'architecture' : 'CLXR_combined',
        'activation_function' : 'Tanh',
        'loss_type' : 'logloss',
        'optimize_for' : 'pos_at_20',
        'epochs':epochs
        })
    
    loader = torch.utils.data.DataLoader(train_array, batch_size=batch_size, shuffle=True)
    num_batches = int(np.ceil(train_array.shape[0] / batch_size))
    num_of_bins = 10

    ## A list for storing all bin index in all epochs. To select the best value among epochs
    tot_bin_indx=[]
    
    ## A list for storing all flipping values (number of users that reversion occurs) in all epochs. To select the best value among epochs
    flip_total=[]
    
    ## A list for storing perturbations in all epochs. To select the best value among epochs
    total_items= []
    
    train_losses = []

    recommender.eval()

    num_features = num_items_dict[data_name]

    explainer = Explainer(num_features, num_items, explainer_hidden_size).to(device) 

    optimizer_comb = torch.optim.Adam(explainer.parameters(), learning_rate)
    loss_func = CLXR_loss(lambda_pos, lambda_neg,lambda_cmp_m1, lambda_cmp_m3, lambda_cmp_neg, alpha)

    print('======================== new run ========================')

    for epoch in range(epochs):
        if epoch%15 == 0 and epoch>0: # decrease learning rate every 15 epochs
            learning_rate*= 0.1
            optimizer_comb.lr = learning_rate

        
        train_loss=0
        explainer.train()
        
        for batch_index, samples in enumerate(loader):
            # prepare data for explainer:
            user_tensors = torch.tensor(samples[:, :-1], dtype=torch.float, device=device)

            user_ids = samples[:,-1]
            ### target items batch for training explainer (i1 is the target item id)
            i1 = np.array([targ_train[int(x)] for x in user_ids])  

            ### contrastive items batch for training explainer (i2 is the comparative item id)
            i2 = np.array([np.random.choice(cmp_train[int(x)]) for x in user_ids])

        
            i1_vectors = items_array[i1]
            i2_vectors = items_array[i2]
            
            ### converting to a tensor
            i1_tensors = torch.Tensor(i1_vectors).to(device)
            i2_tensors = torch.Tensor(i2_vectors).to(device)
            n = user_tensors.shape[0]

            # zero grad:
            optimizer_comb.zero_grad()
            # forward:
            #### scores for the target item
            m1 = explainer(user_tensors, i1_tensors)
            
            #### scores for the contrastive item
            m2= explainer(user_tensors, i2_tensors)
            
            #### comparative scores (m3)
            m3 =(1-m2)*m1

            # calculate loss
            comb_loss = loss_func(user_tensors, i1_tensors , i2_tensors, i1, i2, m1, m2, m3)
    
            train_loss += comb_loss*n

            # back propagation
            comb_loss.backward()
            optimizer_comb.step()

        

        torch.save(explainer.state_dict(), Path(checkpoints_path, f'CLXR_{data_name}_{recommender_name}_{trial.number}_{epoch}_{explainer_hidden_size}_{lambda_pos}_{lambda_neg}.pt'))
        
        explainer.eval()
        
        ## Storing bin indexes in each epoch
        bin_indx=[]
        
        ## storing perturbations in each epoch
        tot_items=[]
        
        for j in range(random_sampled_array.shape[0]):

            user_id = random_sampled_array[j][-1]
            user_tensor = torch.Tensor(random_sampled_array[j][:-1]).to(device)
            
            ## Target item for testing dataset
            i1 = targ_test[user_id]
            
            ## Comparative item for testing dataset
            i2=np.random.choice(cmp_test[user_id])

            i1_vector = items_array[i1]
            i2_vector = items_array[i2]

            i1_tensor = torch.Tensor(i1_vector).to(device)
            i2_tensor = torch.Tensor(i2_vector).to(device)
            
            p, ind= calculate_pos_neg_k(user_tensor, i1, i2, i1_tensor, i2_tensor, num_of_bins, explainer, k=10)

            if p is not None:
                
                bin_indx.append(ind)
                tot_items.append(p)
            
              

        flip_total.append(len(bin_indx))  
    
        ## average of total items for each epoch
        items_avg=np.mean(tot_items)
        ## creating a list for storing values of "total_items_avg" in each epoch
        total_items.append(items_avg)

        
        tot_bin_indx.append(np.mean(bin_indx))
        
    
        print(f'Finished epoch {epoch} with  MPRR(raw) {np.mean(tot_items)}, MPRR(%) {np.mean(bin_indx)*10},'
        f'and Coverage (%) {len(bin_indx)*100/num_of_rand_users}')

    
        
    print(f'Stop at trial with learning rate {learning_rate}, batch size={batch_size},' 
    f'explainer hidden size={explainer_hidden_size}, lambda_pos = {lambda_pos}, '
    f'lambda_neg = {lambda_neg}, alpha_parameter = {alpha}, lambda_cmp_m1 = {lambda_cmp_m1}, '
    f'lambda_cmp_m3 = {lambda_cmp_m3}, lambda_cmp_neg ={lambda_cmp_neg} .' 
    f'Best results at epoch {np.argmin(total_items)} with MPRR (raw) {np.min(total_items)},'
    f'MPRR (%)  {tot_bin_indx[np.argmin(total_items)]*10}'
    f'and Coverage with value {flip_total[np.argmin(total_items)]*100/num_of_rand_users}')    
    
    
    return np.max(flip_total)*100/ num_of_rand_users # return the best total items value in this trial

### Save logs in text file, optimize using Optuna

In [ ]:
logger = logging.getLogger()

logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler(f"{data_name}_{recommender_name}_explainer_training.log", mode="w"))

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

study = optuna.create_study(direction='minimize')

logger.info("Start optimization.")
study.optimize(clxr_training, n_trials=50)   ## For finding the best hyperparameters.

with open(f"{data_name}_{recommender_name}_explainer_training.log") as f:
    assert f.readline().startswith("A new study created")
    assert f.readline() == "Start optimization.\n"
    
    
# Print best hyperparameters and corresponding metric value
print("Best hyperparameters: {}".format(study.best_params))
print("Best metric value: {}".format(study.best_value))